In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('classification.csv', index_col='Unnamed: 0')
df['target'] = (df['target']=='Yes').astype('int')
X_train, X_test, y_train, y_test = train_test_split(df[['col1', 'col2']], df['target'], 
                                                    test_size=0.33, random_state=42)

In [88]:
import numpy as np

class Sigmoid:
    
    def evaluate0(x):
        return 1 / (1 + np.exp(-x))
    
    def evaluate1(x):
        if type(x) is np.ndarray:
            return np.diag(np.exp(-x) / np.square(1 + np.exp(-x)))
        elif type(x) in [int,float]:
            return np.exp(-x) / np.square(1 + np.exp(-x))
    
    
class Relu:
    
    def ev0(x):
        if x > 0:
            return x
        return 0
    
    def ev1(x):
        if x > 0:
            return 1
        return 0
    
    def evaluate0(x):
        if type(x) is np.ndarray:
            return np.array(list(map(Relu.ev0,x)))
        elif type(x) in [float, int]:
            return Relu.ev0(x)
        
    def evaluate1(x):
        if type(x) is np.ndarray:
            return np.diag(np.array(list(map(Relu.ev1,x))))
        elif type(x) in [float, int]:
            return Relu.ev1(x)
        
        
class Random:
    
    def MakeVector(rows:int):
        return np.random.sample(rows)
    
    def MakeMatrix(rows:int, columns:int):
        return np.random.sample((rows, columns))
    
    
class NetLayer:
    
    def __init__(self, rows:int,columns:int, function):
        self.__A = Random.MakeMatrix(rows, columns)
        self.__b = Random.MakeVector(rows)
        self.__function = function
        
    def print_weights(self):
        print(self.__A)
        
    def print_bias(self):
        print(self.__b)
        
    def shift_A(self, A):
        self.__A -= A
        
    def shift_b(self, b):
        self.__b -= b
        
    def predict(self, x):
        return self.__function.evaluate0(self.__A.dot(x) + self.__b)
    
    def count_grad_A(self, x, u):
        return np.outer(self.__function.evaluate1(self.__A.dot(x) + self.__b).dot(u), x)
    
    def count_grad_b(self, x, u):
        return self.__function.evaluate1(self.__A.dot(x) + self.__b).dot(u)
    
    def count_grad_x(self, x, u):
        return u.dot(self.__function.evaluate1(self.__A.dot(x) + self.__b)).dot(self.__A)
    

class LossFunction:
    
    def count_loss_function(self, data, z):
        result = 0
        for i in range(len(z)):
            result += (z[i] - data[i]['Output']).dot(z[i] - data[i]['Output'])
        return result / len(z)
    
    def count_starting_gradient(self,data,z):
        return 2 * (z - data['Output'])
    
class NeuralNetwork:
    
    def __init__(self, dim1:int, dim2:int, dim3:int, function):
        self.__NL1 = NetLayer(dim2, dim1, function)
        self.__NL2 = NetLayer(dim3, dim2, function)
        self.__LF = LossFunction()
        
    def train(self, data):
        z1 = []
        z2 = []
        u1 = []
        u2 = []
        for i in range(len(data)):
            z1.append(self.__NL1.predict(data[i]['Input']))
            z2.append(self.__NL2.predict(z1[i]))
            u2.append(self.__LF.count_starting_gradient(data[i], z2[i]))
            u1.append(self.__NL2.count_grad_x(z1[i], u2[i]))
        n = 1 
        j = 0
        v_A2 = [0., 0.]
        v_A1 = [0., 0.]
        v_b2 = [0., 0.]
        v_b1 = [0., 0.]
        while self.__LF.count_loss_function(data, z2) > 0.2:
            print(self.__LF.count_loss_function(data, z2))
            for i in range(len(data)):
                z1[i] = self.__NL1.predict(data[i]['Input'])
                z2[i] = self.__NL2.predict(z1[i])
                u2[i] = self.__LF.count_starting_gradient(data[i], z2[i])
                u1[i] = self.__NL2.count_grad_x(z1[i], u2[i])
            grad_A2 = np.zeros((len(z2[0]), len(z1[0])))
            grad_b2 = np.zeros(len(z2[0]))
            grad_A1 = np.zeros((len(z1[0]), len(data[0]['Input'])))
            grad_b1 = np.zeros(len(z1[0]))
            for i in range(len(data)):
                grad_A2 += self.__NL2.count_grad_A(z1[i], u2[i])
                grad_b2 += self.__NL2.count_grad_b(z1[i], u2[i])
                grad_A1 += self.__NL1.count_grad_A(data[i]['Input'], u1[i])
                grad_b1 += self.__NL1.count_grad_b(data[i]['Input'], u1[i])
            grad_A2 = grad_A2 / len(data)
            grad_b2 = grad_b2 / len(data)
            grad_A1 = grad_A1 / len(data)
            grad_b1 = grad_b1 / len(data)
            if j == 0:
                v_A2[0] = grad_A2
                v_b2[0] = grad_b2
                v_A1[0] = grad_A1
                v_b1[0] = grad_b1
            if j != 0:
                v_A2[j % 2] = 0.9 * v_A2[(j - 1) % 2] + grad_A2 / n
                v_b2[j % 2] = 0.9 * v_b2[(j - 1) % 2] + grad_b2 / n
                v_A1[j % 2] = 0.9 * v_A1[(j - 1) % 2] + grad_A1 / n
                v_b1[j % 2] = 0.9 * v_b1[(j - 1) % 2] + grad_b1 / n
            self.__NL2.shift_A(v_A2[j % 2])
            self.__NL2.shift_b(v_b2[j % 2])
            self.__NL1.shift_A(v_A1[j % 2])
            self.__NL1.shift_b(v_b1[j % 2])
            if j % 100 == 0 and n < 30:
                n += 1 
            j += 1
    def predict(self, x):
        return self.__NL2.predict(self.__NL1.predict(x))

In [31]:
s=Sigmoid
s.evaluate1(np.array([0,1]))

array([[0.25      , 0.        ],
       [0.        , 0.19661193]])

In [53]:
r=Relu
r.evaluate1(np.array([-4,10]))

array([[0, 0],
       [0, 1]])

In [58]:
Random.MakeMatrix(3,5)

array([[0.80810869, 0.94016195, 0.35361447, 0.930358  , 0.98639779],
       [0.81279673, 0.42968184, 0.39875573, 0.1021755 , 0.30224717],
       [0.72935014, 0.12791058, 0.89118214, 0.88020474, 0.72270238]])

In [123]:
NL=NetLayer(2,3,Sigmoid)

In [68]:
A=np.array([[-10, -10, -10],
 [-10, -10, -10]])
NL.shift_A(A)
NL.print_weights()

[[10. 10. 10.]
 [10. 10. 10.]]


In [108]:
NL.print_weights()

[[0.20367809 0.85214081 0.59869458]
 [0.03928164 0.93976695 0.47034487]]


In [109]:
NL.print_bias()

[0.77452469 0.06268422]


In [84]:
x=np.array([0,2,-3])
NL.predict(x)

array([0.0895792 , 0.55046554])

In [110]:
u=np.array([1,-2])
NL.count_grad_b(x,u)

array([ 0.22299037, -0.46632515])

In [111]:
NL.count_grad_A(x,u)

array([[ 0.        ,  0.44598073, -0.6689711 ],
       [-0.        , -0.9326503 ,  1.39897545]])

In [115]:
NL.count_grad_x(x,u)

array([-0.24376991, -0.10961422, -0.14425359])

In [124]:
data=({'Input':np.array([2,1,3]),'Output':np.array([0.2,0.1,0.3])},{'Input':np.array([1,-1,0]),'Output':np.array([0.1,-0.1,0])})
z=(np.array([0.3,0.1,-0.3]),np.array([0.1,0.1,-0.4]))
LossFunction.count_loss_function(data,z)

0.28500000000000003

In [121]:
data={'Input':np.array([2,1,3]), 'Output':np.array([0.2,0.1,0.3])}
z=np.array([0.1,0.1,-0.4])
LossFunction.count_starting_gradient(data,z)

array([-0.2,  0. , -1.4])

In [26]:
data = []
for i in range(len(X_train)):
    data.append({'Input':np.array(X_train.iloc[i]), 'Output':np.array(y_train.iloc[i])})

In [ ]:
Sigma = Sigmoid
NN = NeuralNetwork(2,2,1,Sigma)
NN.train(data)

0.3587398451259916
0.3587398451259916
0.3462884146104537
0.32818075697548926
0.3052557095721496
0.28044723153793105
0.2597739641734529
0.2502401818024882
0.2539333878195276
0.2655964288955525
0.2777264009013657
0.28546541789221597
0.2870972184185841
0.2828845467507892
0.2743544447964117
0.26408843462599024
0.25529500851161224
0.2506178429485149
0.250747118204292
0.25422016563217287
0.25862603520892075
0.2619354077499632
0.2630744204051451
0.2618999992251761
0.25898192743475057
0.2553502731588551
0.25217453672103085
0.2503593161791538
0.25019869334432066
0.25130937782312374
0.252896297952805
0.25415765788750655
0.25459103264583954
0.25410394918170076
0.25295892617591953
0.2516166832288032
0.25053809047189285
0.2500112726477542
0.2500689556134611
0.250522648987571
0.2510796039678755
0.25147408049255665
0.25155702689434295
0.25132365500050974
0.2508864463263872
0.2504140532589357
0.25006098528993326
0.24991298242311485
0.24996628512591715
0.25014451759424766
0.2503406864882488
0.250463089

0.24813160340696308
0.24811418663021498
0.24809664648944937
0.2480789782225439
0.24806117743854403
0.24804324008146258
0.24802516239766076
0.24800694090646896
0.2479885723737288
0.2479700537879747
0.2479513823389948
0.24793255539854184
0.24791357050298177
0.24789442533768719
0.2478751177230096
0.24785564560167517
0.24783600702745645
0.24781620015500397
0.2477962232307162
0.2477760745845508
0.24775575262267396
0.24773525582088016
0.2477145827186897
0.24769373191406538
0.24767270205868733
0.24765149185372423
0.2476301000460541
0.24760852542489561
0.24758676681880146
0.24756482309297886
0.2475426931469091
0.24752037591223416
0.247497870350884
0.24747517545342038
0.2474522902375778
0.24742921374698146
0.2474059450500235
0.24738248323888473
0.24735882742868767
0.24733497675676588
0.2473109303820381
0.24728668748448177
0.247262247264693
0.24723760894352304
0.24721277176178855
0.24718773498004373
0.2471624978784146
0.2471370597564842
0.24711141993322416
0.24708557774697285
0.24705953255545415

0.23522366934462305
0.23520218002753168
0.23518074146833304
0.23515935295632717
0.23513801384569488
0.23511672354891897
0.2350954815308657
0.23507428730345437
0.23505314042086595
0.23503204047523152
0.23501098709275017
0.23498997993020265
0.23496901867180808
0.23494810302640112
0.23492723272489135
0.23490640751797665
0.2348856271740874
0.23486489147753803
0.23484420022686336
0.23482355323332163
0.2348029503195525
0.23478239131836282
0.23476187607164384
0.23474140442938846
0.2347209762488174
0.23470059139358337
0.2346802497330678
0.23465995114173507
0.2346396954985667
0.2346194826865409
0.23459931259217368
0.23457918510510317
0.2345591001177171
0.234539057524817
0.23451905722332184
0.23449909911199734
0.23447918309121585
0.23445930906274065
0.23443947692953365
0.2344196865955814
0.23439993796574385
0.2343802309456142
0.23436056544139594
0.23434094135979366
0.23432135860791795
0.2343018170931946
0.23428231672329078
0.23426285740604505
0.23424343904940648
0.23422406156138315
0.23420472484

0.2297531578143834
0.22974517681388068
0.22973720378264353
0.22972923867122017
0.22972128143324863
0.229713332025155
0.22970539040587767
0.22969745653662352
0.22968953038064308
0.22968161190303293
0.22967370107055413
0.22966579785147176
0.2296579022154086
0.22965001413321456
0.22964213357684696
0.22963426051926633
0.22962639493434092
0.22961853679675817
0.2296106860819505
0.22960284276602183
0.2295950068256884
0.2295871782382205
0.22957935698139167
0.22957154303343175
0.22956373637299057
0.22955593697909224
0.22954814483111072
0.22954035990873398
0.22953258219193826
0.22952481166096508
0.2295170482962982
0.22950929207864323
0.22950154298891035
0.2294938010081993
0.22948606611778358
0.22947833829909683
0.22947061753372466
0.22946290380338827
0.22945519708994108
0.2294474973753563
0.22943980464172004
0.2294321188712236
0.22942444004616
0.22941676814891557
0.229409103161965
0.2294014450678683
0.22939379384926523
0.22938614948887295
0.22937851196947973
0.22937088127394653
0.229363257385199

0.22721377033262538
0.22720900989363657
0.2272042513565204
0.22719949471548276
0.22719473996496017
0.2271899870995991
0.22718523611423375
0.22718048700386712
0.22717573976365887
0.2271709943889045
0.2271662508750277
0.2271615092175654
0.22715676941215587
0.2271520314545339
0.2271472953405169
0.2271425610660002
0.22713782862694734
0.2271330980193876
0.2271283692394072
0.22712364228314388
0.22711891714678528
0.22711419382656153
0.22710947231874393
0.22710475261963967
0.22710003472558965
0.2270953186329658
0.22709060433816958
0.2270858918376267
0.22708118112778972
0.22707647220513105
0.22707176506614546
0.22706705970734628
0.22706235612526585
0.22705765431645294
0.22705297897010537
0.22704832759521398
0.2270436979496743
0.22703908801537923
0.2270344959758071
0.22702992019585597
0.22702535920369096
0.2270208116744147
0.227016276415368
0.22701175235289997
0.22700723852046675
0.22700273404791674
0.22699823815185147
0.22699375012694611
0.22698926933814448
0.22698479521362755
0.226980327238493

...

0.2203358817801285
0.2203336310564453
0.22033138040943978
0.2203291298391234
0.22032687934551104
0.22032462892861476
0.22032237858844889
0.22032012832502704
0.22031787813836134
0.22031562802846713
0.2203133779953567
0.2203111280390437
0.22030887815954273
0.22030662835686682
0.2203043786310296
0.22030212898204518
0.22029987940992765
0.2202976299146902
0.22029538049634698
0.2202931311549124
0.22029088189039966
0.22028863270282348
0.22028638359219802
0.22028413455853646
0.22028188560185394
0.2202796367221641
0.2202773879194821
0.220275139193821
0.2202728905451956
0.22027064197362078
0.22026839347911037
0.22026614506167924
0.22026389672134103
0.22026164845811144
0.22025940027200444
0.2202571521630347
0.22025490413121762
0.2202526561765664
0.22025040829909712
0.2202481604988239
0.22024591277576186
0.22024366512992552
0.22024141756133028
0.22023917006999075
0.2202369226559223
0.22023467531913948
0.22023242805965768
0.22023018087749205
0.22022793377265756
0.2202256867451691
0.2202234397950432

0.21940418790191357
0.21940197075453435
0.21939975369247283
0.21939753671575707
0.21939531982441374
0.21939310301846893
0.21939088629795023
0.21938866966288426
0.21938645311329805
0.21938423664921808
0.21938202027067202
0.21937980397768628
0.21937758777028865
0.21937537164850515
0.21937315561236398
0.2193709396618908
0.21936872379711347
0.2193665080180595
0.21936429232475577
0.21936207671722913
0.21935986119550674
0.21935764575961564
0.21935543040958458
0.2193532151454391
0.2193509999672069
0.21934878487491566
0.21934656986859208
0.2193443549482642
0.21934214011395925
0.21933992536570407
0.21933771070352695
0.21933549612745418
0.21933328163751442
0.21933106723373433
0.21932885291614193
0.2193266386847641
0.21932442453962886
0.2193222104807637
0.21931999650819597
0.21931778262195376
0.21931556882206407
0.219313355108555
0.21931114148145436
0.2193089279407895
0.21930671448658792
0.2193045011188781
0.21930228783768718
0.21930007464304305
0.21929786153497294
0.2192956485135061
0.2192934355

0.21848956303320535
0.21848738392022657
0.21848520490526768
0.21848302598836292
0.21848084716954516
0.21847866844884736
0.21847648982630394
0.21847431130194683
0.21847213287581124
0.2184699545479286
0.21846777631833375
0.21846559818705968
0.21846342015413966
0.21846124221960733
0.21845906438349547
0.21845688664583804
0.21845470900666897
0.21845253146602123
0.2184503540239279
0.2184481766804234
0.21844599943553966
0.21844382228931197
0.21844164524177304
0.21843946829295585
0.21843729144289495
0.21843511469162347
0.2184329380391745
0.21843076148558183
0.2184285850308795
0.21842640867510058
0.21842423241827913
0.21842205626044753
0.218419880201641
0.2184177042418923
0.21841552838123493
0.2184133526197023
0.21841117695732856
0.21840900139414768
0.21840682593019237
0.21840465056549657
0.2184024753000943
0.21840030013401934
0.21839812506730497
0.21839595009998533
0.2183937752320925
0.21839160046366235
0.21838942579472712
0.21838725122532132
0.2183850767554783
0.21838290238523222
0.2183807281

0.2175919190673412
0.2175897836879012
0.2175876484208557
0.21758551326624098
0.2175833782240912
0.21758124329444167
0.2175791084773278
0.21757697377278473
0.21757483918084733
0.21757270470155116
0.21757057033493055
0.21756843608102117
0.21756630193985851
0.21756416791147723
0.2175620339959123
0.21755990019319907
0.2175577665033727
0.21755563292646857
0.21755349946252134
0.21755136611156603
0.2175492328736384
0.2175470997487732
0.21754496673700585
0.21754283383837145
0.21754070105290432
0.2175385683806402
0.21753643582161433
0.21753430337586135
0.21753217104341782
0.2175300388243165
0.21752790671859443
0.21752577472628618
0.21752364284742665
0.21752151108205078
0.2175193794301944
0.2175172478918922
0.21751511646717908
0.2175129851560908
0.21751085395866185
0.21750872287492795
0.21750659190492302
0.21750446104868384
0.21750233030624447
0.21750019967763956
0.2174980691629057
0.217495938762077
0.21749380847518907
0.21749167830227603
0.21748954824337408
0.21748741829851848
0.217485288467743

0.21671163414088013
0.2167095486545806
0.2167074632951119
0.2167053780625078
0.21670329295680163
0.21670120797802728
0.2166991231262185
0.21669703840140847
0.21669495380363127
0.21669286933292
0.21669078498930874
0.21668870077283112
0.21668661668351996
0.21668453272140983
0.21668244888653365
0.21668036517892503
0.21667828159861807
0.21667619814564532
0.21667411482004129
0.21667203162183893
0.2166699485510721
0.2166678656077739
0.21666578279197843
0.2166637001037188
0.2166616175430285
0.21665953510994113
0.2166574528044904
0.21665537062670964
0.21665328857663183
0.2166512066542906
0.21664912485972027
0.21664704319295355
0.21664496165402347
0.2166428802429643
0.21664079895980962
0.2166387178045918
0.21663663677734502
0.21663455587810265
0.216632475106898
0.21663039446376406
0.21662831394873508
0.21662623356184357
0.21662415330312298
0.2166220731726074
0.21661999317032934
0.2166179132963232
0.2166158335506215
0.21661375393325724
0.21661167444426427
0.21660959508367653
0.21660751585152613


In [83]:
y_pred = []
for i in range(len(X_test)):
    y_pred.append(NN.predict(np.array(X_test.iloc[i]))[0])

In [87]:
from sklearn.metrics import roc_auc_score
roc_auc_score(np.array(y_test), np.array(y_pred))

0.5917824074074074